In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
# USED ONLY FOR TESTING
# NOT GOOD FOR SEQ TO SEQ, IT IS NOT JUST CLASSIFICATION
# IMPLEMENT WHAT THEY DID IN TASK WHERE WE GOT THE IDEA

# CONCLUSIONS: TOO MUCH FALSE NEGATIVES - BAD RECALL
#              BIGGER LATENT_TIM -> BETTER RESULTS -- LAT_DIM-F1 : 13-68, 64-70 256-73
def simple_evaluation(y_pred, y_test):
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for i, pred_arr in enumerate(y_pred):
    for j, pred_value in enumerate(pred_arr):
      predicted = pred_value
      real = y_test[i][j]
      if predicted and real:
        tp = tp + 1
      elif not(predicted) and real:
        fn = fn + 1
      elif predicted and not(real):
        fp = fp + 1
      elif not(predicted) and not(real):
        tn = tn + 1
    # break

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)

  print("PRECISION: " + str(precision))
  print("RECALL: " + str(recall))
  print("F1: " + str(f1))
  print("-"*20)
  print("tp: " + str(tp))
  print("tn: " + str(tn))
  print("fp: " + str(fp))
  print("fn: " + str(fn))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/mreze/data/data/full_sequences.pkl')
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# Train model only to recognize blender/(one type of) events. 
training = data[data['event_label'] == 'Blender']
X = np.array(training.mfccs.tolist())
y = np.array(training.combine_squence_per_filename_and_event.tolist())

X = np.einsum('ijk->ikj', X)
training_filenames = training.filename.unique()

In [ ]:
# Use negative samples to train as well
training_false = data[(data['event_label'] != 'Blender') & (~data['filename'].isin(training_filenames))]
X_false = np.array(training_false.mfccs.tolist())
y_false = np.array(training_false.combine_squence_per_filename_and_event.tolist())

X_false = np.einsum('ijk->ikj', X_false)

In [ ]:
shape_of_y_false = y_false.shape
y_false = np.asarray([[0]*shape_of_y_false[1]]*shape_of_y_false[0])
y_false.shape

(2942, 431)

In [ ]:
Y_new = np.concatenate((y, y_false), axis=0)
Y_new.shape

(3378, 431)

In [ ]:
X_new = np.concatenate((X, X_false), axis=0)
X_new.shape

(3378, 431, 13)

In [ ]:
# no comment
X = X_new
y = Y_new

In [ ]:
indices = np.arange(X.shape[0])

# split the dataset 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)

In [ ]:
def create_output_seq(y_orig):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    is_one = False
    new_sequence_beginning = []
    new_sequence_end = []
    to_assign_beginning = 0
    to_assign_end = 0
    for label in sequence:
      if label == 1:
        if is_one:
          to_assign_beginning = 0
        else:
          to_assign_beginning = 1
        is_one = True
      else:
        if is_one:
          new_sequence_end[len(new_sequence_end)-1] = 1
          to_assign_end = 0
        else:
          to_assign_end = 0
        is_one = False
      new_sequence_beginning.append(to_assign_beginning)
      new_sequence_end.append(to_assign_end)
    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
def create_output_seq_expanded(y_orig, expansion_num):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    new_sequence_beginning = [0] * len(sequence)
    new_sequence_end = [0] * len(sequence)
    indexes_beginning = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 0 and sequence[i+1] == 1] 
    indexes_end = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 1 and sequence[i+1] == 0] 

    if sequence[len(sequence)-1] == 1:
      indexes_end.append(len(sequence))
    for index in indexes_beginning:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_beginning[start:end] = [1] * (end - start)

    for index in indexes_end:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_end[start:end] = [1] * (end - start)

    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
# Prepare output sequences with PADDED indicators for beginning & end (for training data)
# Example:
# [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0] will become
# [0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] (for beggining) and
# [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0] (for beggining)
y_new_beginning_exp, y_new_end_exp = create_output_seq_expanded(y_train, 2)
# ind = 2799
# print(y_train[ind])
# print(80*'-')
# print(y_new_beginning_exp[ind])
# print(80*'-')
# print(y_new_end_exp[ind])

In [ ]:
# Optional -> to separate sequences into smaller ones -> more data. 
def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  return ([lst[i:i + n] for i in range(0, len(lst), n)])


def split_sequences_into_seconds(X, y, seconds = 1):
  sequence_len = X.shape[1]
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)

  X_new = []
  y_new = []
  for i, sequence in enumerate(X):
    X_splits = chunks(sequence, ticks_per_chosen_second)
    y_splits = chunks(y[i], ticks_per_chosen_second)
    x_false_instances = np.array([]).reshape(0,X_splits[0].shape[1])
    for j, X_split in enumerate(X_splits):
      X_temp = X_split
      y_temp = y_splits[j]
      if len(x_false_instances) < ticks_per_chosen_second:
        false_indices = np.where(~y_temp)
        x_false_instances = np.append(x_false_instances, X_split[false_indices], axis=0)

      diff = ticks_per_chosen_second - len(X_temp)
      if diff != 0:
        y_temp = np.append(y_temp, [0] * diff)
        X_temp = np.append(X_temp, x_false_instances[0: diff], axis=0)
        
      X_new.append(X_temp)
      y_new.append(y_temp)


  return np.asarray(X_new), np.asarray(y_new)

X_train_splitted, y_new_beginning_exp_splitted = split_sequences_into_seconds(X_train, y_new_beginning_exp, 5)
X_train_splitted, y_new_end_exp_splitted = split_sequences_into_seconds(X_train, y_new_end_exp, 5)

In [ ]:
print(X_train.shape)
print(X_train_splitted.shape)

(3061, 431, 13)
(6122, 216, 13)


In [ ]:
# Prepare output sequences with PADDED indicators for beginning & end (for test data)
y_test_new_beginning_exp, y_test_new_end_exp = create_output_seq_expanded(y_test, 2)

In [ ]:
# Optional
# y_test_new_beginning_exp, y_test_new_end_exp = create_output_seq_expanded(y_test, 2)
# X_test_splitted, y_test_new_beginning_exp_splitted = split_sequences_into_seconds(X_test, y_test_new_beginning_exp, 5)
# X_test_splitted, y_test_new_end_exp_splitted = split_sequences_into_seconds(X_test, y_test_new_end_exp, 5)

In [ ]:
input_seq_len = 431
output_shape = 431
dense_layer_size = 150
num_of_features = 13
# output_len = len(data)

batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.

In [ ]:
# Define architecture of autoencoder model. 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

def create_encoder_decoder(input_seq_len = 431, num_of_features = 13):
  # Define an input sequence and process it.

  encoder_inputs = keras.Input(shape=(input_seq_len, num_of_features))
  encoder = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
  encoder_outputs, state_h = encoder(encoder_inputs)

  encoder_states = [state_h]

  decoder_beg = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_beg_outputs_0 = decoder_beg(encoder_outputs)

  decoder_beg_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_beg_outputs_2 = decoder_beg_2(decoder_beg_outputs_0)

  decoder_end = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_end_outputs_0 = decoder_end(encoder_outputs)

  decoder_end_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_end_outputs_2 = decoder_end_2(decoder_end_outputs_0)

  return encoder_inputs, decoder_beg_outputs_2, decoder_end_outputs_2

In [ ]:
encoder_inputs, decoder_outputs_beg, decoder_outputs_end = create_encoder_decoder(X_train.shape[1])
model = keras.Model(encoder_inputs, [decoder_outputs_beg, decoder_outputs_end])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 431, 13)]    0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 431, 128), ( 54912       input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 431, 150)     19350       gru[0][0]                        
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 431, 150)     19350       gru[0][0]                        
______________________________________________________________________________________________

In [ ]:
from keras.metrics import BinaryAccuracy, Precision, Recall, AUC

METRICS = [
      Precision(name='precision'),
      Recall(name='recall'),
]

model.compile(
    optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=METRICS
)
checkpoint_filepath = '/content/drive/MyDrive/mreze/models/seq_to_seq'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_dense_3_precision',
    mode='max',
    save_best_only=True)


model.fit(
    [X_train],
    [y_new_beginning_exp, y_new_end_exp],
    batch_size=batch_size,
    epochs=400,
    validation_data=(X_test, [y_test_new_beginning_exp, y_test_new_end_exp]),
    callbacks=[model_checkpoint_callback]
)

Epoch 1/400
22/22 [==============================] - 4s 87ms/step - loss: 0.0138 - dense_1_loss: 0.0085 - dense_3_loss: 0.0052 - dense_1_precision: 0.5610 - dense_1_recall: 0.0280 - dense_3_precision: 0.5949 - dense_3_recall: 0.0287 - val_loss: 0.0218 - val_dense_1_loss: 0.0117 - val_dense_3_loss: 0.0102 - val_dense_1_precision: 0.5325 - val_dense_1_recall: 0.0257 - val_dense_3_precision: 0.5325 - val_dense_3_recall: 0.0251
Epoch 2/400
22/22 [==============================] - 1s 43ms/step - loss: 0.0132 - dense_1_loss: 0.0083 - dense_3_loss: 0.0050 - dense_1_precision: 0.5425 - dense_1_recall: 0.0242 - dense_3_precision: 0.5420 - dense_3_recall: 0.0243 - val_loss: 0.0216 - val_dense_1_loss: 0.0121 - val_dense_3_loss: 0.0094 - val_dense_1_precision: 0.5333 - val_dense_1_recall: 0.0238 - val_dense_3_precision: 0.5333 - val_dense_3_recall: 0.0236
Epoch 3/400
22/22 [==============================] - 1s 42ms/step - loss: 0.0129 - dense_1_loss: 0.0082 - dense_3_loss: 0.0047 - dense_1_precisi

In [ ]:
y_test_new_beginning_exp.shape

(766, 431)

In [ ]:
test = [i for i in range(len(y_train)) if len(np.nonzero(y_train[i])[0]) > 0]
test[6]

58

In [ ]:
# Manually check and visually compare/show predictions. 
num = 58
# X_test_splitted, [y_test_new_beginning_exp_splitted, y_test_new_end_exp_sp
predicted = model.predict([np.array([X_train[num]])])

print(np.round(np.concatenate(predicted[0][0], axis=0), 3))
# print(np.concatenate(predicted[0][0], axis=0))
# print(np.round(predicted[0][0], 3))
# print(predicted[0][0])
print("-------------------------------- BEGINNINGS --------------------------------")
predicted_beg_test = np.round(np.concatenate(predicted[0][0], axis=0), 0).astype(int)
print(predicted_beg_test)
# print(y_new_beginning[num])
print("-------------------------------- ENDINGS --------------------------------")
predicted_end_test = np.round(np.concatenate(predicted[1][0], axis=0), 0).astype(int)
print(predicted_end_test)
# print(y_new_end[num])
print("-------------------------------- ORIGINAL --------------------------------")

print(y_train[num])

[0.    0.002 0.001 0.001 0.001 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.001 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0

In [ ]:
# X_test_splitted, y_test_splitted = split_sequences_into_seconds(X_test, y_test, 5)

In [ ]:
model.load_weights(checkpoint_filepath)
predicted_beg, predicted_end = model.predict(X_test)

In [ ]:
predicted_beg_rounded = [np.round(np.concatenate(i, axis=0), 0).astype(int) for i in predicted_beg]
predicted_end_rounded = [np.round(np.concatenate(i, axis=0), 0).astype(int) for i in predicted_end]

In [ ]:
simple_evaluation(predicted_beg_rounded,y_new_beginning_exp)
simple_evaluation(predicted_end_rounded,y_new_end_exp)


ZeroDivisionError: ignored

In [ ]:
# When expanded/padded Y_train is used
def combine_beginning_end_exp(y_beg, y_end):
  result_begining = []
  result_ending = []

  for j in range(0, len(y_beg)):
    y_beg_curr = np.asarray(y_beg[j])
    y_end_curr = np.asarray(y_end[j])
    y_final = []

    y_beg_indices = np.where(y_beg_curr == 1)[0]
    y_end_indices = np.where(y_end_curr == 1)[0]


    y_beg_groups = []
    group = []
    for i, value in enumerate(y_beg_indices):
      if i == 0:
        group.append(value)
        continue
      if (value - y_beg_indices[i-1]) > 2 or i == len(y_beg_indices) - 1:
        if i == len(y_beg_indices) - 1:
          group.append(value)
        y_beg_groups.append(group.copy())
        group = []
        group.append(value)
      else:
        group.append(value)

    y_end_groups = []
    group = []
    for i, value in enumerate(y_end_indices):
      if i == 0:
        group.append(value)
        continue
      if (value - y_end_indices[i-1]) > 2 or i == len(y_end_indices) - 1:
        if i == len(y_end_indices) - 1:
          group.append(value)
        y_end_groups.append(group.copy())
        group = []
        group.append(value)
      else:
        group.append(value)


    new_sequence_beginning = [0] * len(y_beg_curr)
    new_sequence_ending = [0] * len(y_end_curr)

    for group in y_beg_groups:
      start = group[0]
      end = group[len(group)-1]
      new_sequence_beginning[math.floor((start + end) / 2)] = 1

    for group in y_end_groups:
      start = group[0]
      end = group[len(group)-1]
      new_sequence_ending[math.ceil((start + end) / 2)] = 1

    result_begining.append(new_sequence_beginning)
    result_ending.append(new_sequence_ending)

  return combine_beginning_end(result_begining, result_ending)

In [ ]:
def combine_predicted_to_original(y_pred, seconds, sequence_len):
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)
  y_pred_combined = []
  # Num of arrays at the end
  final_size = math.floor(y_pred.shape[0] / num_of_parts_of_sequence)

  print(num_of_parts_of_sequence)
  print(final_size)

  for i in range(0, final_size):
    current_arr = np.array([]).reshape(0,y_pred[0].shape[1])
    for j in range(0, num_of_parts_of_sequence):
      current_arr = np.concatenate((current_arr, y_pred[i * num_of_parts_of_sequence + j])).astype(int)
    y_pred_combined.append(current_arr)
  return y_pred_combined
predicted_end_rounded = np.expand_dims(predicted_end_rounded, axis=-1)
predicted_beg_rounded = np.expand_dims(predicted_beg_rounded, axis=-1)
  
predicted_end_rounded = combine_predicted_to_original(np.asarray(predicted_end_rounded), 5, 431)
predicted_beg_rounded = combine_predicted_to_original(np.asarray(predicted_beg_rounded), 5, 431)

In [ ]:
res = combine_beginning_end_exp(predicted_beg_rounded, predicted_end_rounded)

In [ ]:
# When NOT expanded Y_train is used
def combine_beginning_end(y_beg, y_end):
  result = []
  
  for j in range(0, len(y_beg)):
    y_beg_curr = y_beg[j]
    y_end_curr = y_end[j]
    y_final = []
    was_one = False
    to_append_new = 0
    to_append = 0
    for i in range(0, len(y_beg_curr)):
      if was_one and i == len(y_beg_curr) - 1:
        average_sound_duration = int((i - index_last_beginning) / 2)
        y_final[index_last_beginning + average_sound_duration:i] = [0] * (i - index_last_beginning - average_sound_duration)
        to_append_new = 0
      if y_beg_curr[i] == 1:
        if was_one:
          average_sound_duration = int((i - index_last_beginning) / 2)
          y_final[index_last_beginning + average_sound_duration:i] = [0] * (i - index_last_beginning - average_sound_duration)
        else:
          to_append_new = 1
          was_one = True
        index_last_beginning = i  
        
      if y_end_curr[i] == 1:
        if was_one:
          to_append_new = 0
          was_one = False
      y_final.append(to_append) 
      to_append = to_append_new 
    result.append(y_final)  
  return result 

In [ ]:
y_test_splitted = np.expand_dims(y_test_splitted, axis=-1)
combined_test_y = combine_predicted_to_original(y_test_splitted, 5, 431)
simple_evaluation(res,combined_test_y)

In [ ]:
!pip install sed_eval

In [ ]:
import sed_eval
import dcase_util

def return_onsets_and_offsets_for_sequence(sequence, duration_seconds=10):
  single_tick_in_seconds = duration_seconds / len(sequence)
  onsets = []
  offsets = []

  error_allowance = 5
  error_counter = 0
  in_positive_sequence = 0
  
  for index, tick in enumerate(sequence):
    if tick and not(in_positive_sequence):
      in_positive_sequence = 1
      onsets.append(index * single_tick_in_seconds)
    elif not(tick) and in_positive_sequence:
      if error_counter >= error_allowance:
        error_counter = 0
        in_positive_sequence = 0
        offsets.append((index - error_allowance) * single_tick_in_seconds)
      else:
        error_counter = error_counter + 1
    elif tick and in_positive_sequence and error_counter:
      error_counter = 0
  if in_positive_sequence:
    offsets.append((index) * single_tick_in_seconds)

  return onsets, offsets

def prepare_predicted_sequences_for_evaluation_and_evaluate(event_label, y_pred, I_test, training_data, events_data, return_prepared_data_flag = 0):
  references = []
  estimations = []

  for i, y_predicted in enumerate(y_pred):
    file_index = I_test[i]
    file_name = training_data.iloc[file_index]['filename']

    onsets, offsets = return_onsets_and_offsets_for_sequence(y_predicted, 10)

    for j, onset in enumerate(onsets):
      estimations.append({
          'event_label': event_label,
          'event_onset': onset,
          'event_offset': offsets[j],
          'file': file_name
      })

    reference_events = events_data[(events_data['filename'] == file_name) & (events_data['event_label'] == event_label)]
    for ref_event in np.asarray(reference_events):
      references.append({
          'event_label': event_label,
          'event_onset': ref_event[1],
          'event_offset': ref_event[2],
          'file': file_name
      })

  evaluate_and_print_results(references, estimations)

  if return_prepared_data_flag:
    return references, estimations


def evaluate_and_print_results(references, estimations):
  reference_event_list = dcase_util.containers.MetaDataContainer(
      references
  )

  estimated_event_list = dcase_util.containers.MetaDataContainer(
      estimations
  )

  segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      time_resolution=1.0
  )
  event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      t_collar=0.250,
      evaluate_onset=True,
      evaluate_offset=True,
      percentage_of_length=0.5
  )

  for filename in reference_event_list.unique_files:
      reference_event_list_for_current_file = reference_event_list.filter(
          filename=filename
      )

      estimated_event_list_for_current_file = estimated_event_list.filter(
          filename=filename
      )

      segment_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

      event_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

  # Get only certain metrics
  overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
  print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

  # Or print all metrics as reports
  print(segment_based_metrics)
  print(event_based_metrics)

In [ ]:
events = pd.read_csv("/content/drive/MyDrive/neuronske/data/Synthetic_dataset/synthetic_dataset.csv", delimiter="\t") 
events.head()

In [ ]:
references, estimations = prepare_predicted_sequences_for_evaluation_and_evaluate('Blender', res, I_test, training, events, 1)